In [1]:
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm
else:
    try: import numpy; get_numpy = f"numpy=={numpy.__version__}"
    except: get_numpy = "numpy"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")
    !uv pip install -qqq --upgrade \
        unsloth {get_vllm} {get_numpy} torchvision bitsandbytes xformers
    !uv pip install -qqq {get_triton}
!uv pip install transformers==4.53.0
!uv pip install --no-deps trl==0.22.2

Using Python 3.11.13 environment at: /usr
Resolved 30 packages in 17ms                                         
Uninstalled 2 packages in 84ms
Installed 2 packages in 73ms                                
 - tokenizers==0.22.1
 + tokenizers==0.21.4
 - transformers==4.56.2
 + transformers==4.53.0
Using Python 3.11.13 environment at: /usr
Resolved 1 package in 1ms                                            
Uninstalled 1 package in 4ms
Installed 1 package in 8ms                                  
 - trl==0.23.0
 + trl==0.22.2


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Base model
base_model_name = "unsloth/Qwen3-1.7B-Base"
model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map="cuda:0")
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Load LoRA adapter locally
adapter_path = "/kaggle/input/qwen_grpo_medmcqa/transformers/default/1/grpo-medmcqa-qwen-4b/checkpoint-400"
model = PeftModel.from_pretrained(model, adapter_path)

# Move model to GPU
model.to(device)

2025-09-27 09:37:13.107365: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758965833.461332     175 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758965833.566598     175 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 2048, padding_idx=151654)
        (layers): ModuleList(
          (0-27): 28 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [3]:
!pip -qqq install rouge

In [4]:
from datasets import load_dataset
import re
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge
from vllm import SamplingParams


In [39]:
explanation_start = "<explanation>"
explanation_end   = "</explanation>"
solution_start    = "<answer>"
solution_end      = "</answer>"

system_prompt = f"""The User asks a multiple choice question, and the Assistant answers the question from the given options. Think about the problem, pick the option that answers the question and provide your explanation for choosing that option.

Firstly, provide your answer between {solution_start}{solution_end}.
Then place your explanation between {explanation_start} and {explanation_end}.

Example format:
{solution_start}0{solution_end}
{explanation_start}Your reasoning here{explanation_end}"""

In [50]:
chat_template = \
    "{% if messages[0]['role'] == 'system' %}"\
        "{{ messages[0]['content'] + eos_token }}"\
        "{% set loop_messages = messages[1:] %}"\
    "{% else %}"\
        "{{ '{system_prompt}' + eos_token }}"\
        "{% set loop_messages = messages %}"\
    "{% endif %}"\
    "{% for message in loop_messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ message['content'] }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ message['content'] + eos_token }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}{{ '{reasoning_start}' }}"\
    "{% endif %}"

# Replace with out specific template:
chat_template = chat_template\
    .replace("'{system_prompt}'",   f"'{system_prompt}'")\
    .replace("'{reasoning_start}'", f"'{solution_start}'")
tokenizer.chat_template = chat_template

In [51]:
# simple QA test
#question = "What is the capital of France?"
q = "The most common organism causing urinary tract infections is:\nOptions:\n0. Staphylococcus aureus\n1. Escherichia coli\n2. Pseudomonas aeruginosa\n3. Enterococcus faecalis"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": q},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)

inputs = tokenizer(text, return_tensors="pt").to(device)

"The User asks a multiple choice question, and the Assistant answers the question from the given options. Think about the problem, pick the option that answers the question and provide your explanation for choosing that option.\n\nFirstly, provide your answer between <answer></answer>.\nThen place your explanation between <explanation> and </explanation>.\n\nExample format:\n<answer>0</answer>\n<explanation>Your reasoning here</explanation>The most common organism causing urinary tract infections is:\nOptions:\n0. Staphylococcus aureus\n1. Escherichia coli\n2. Pseudomonas aeruginosa\n3. Enterococcus faecalis<answer>1</answer><explanation>Ans. is 'b' i.e., Escherichia coli</explanation>"

In [47]:
from unsloth import FastLanguageModel
FastLanguageModel.for_inference(model)
inputs = tokenizer(text, return_tensors="pt").to("cuda")
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.1,
        do_sample=True
    )
result = tokenizer.decode(output[0][len(inputs.input_ids[0]):], skip_special_tokens=True)
print("Transformers generation result:")
print(result)

Transformers generation result:
1</answer><explanation>Ans. is 'b' i.e., Escherichia coli</explanation>


In [60]:
def extract_answer_and_explanation(text):
    """Extract answer and explanation from model output"""
    # Look for pattern: number</answer><explanation>text</explanation>
    answer_match = re.search(r'(\d+)</answer>', text)
    explanation_match = re.search(r'<explanation>(.*?)</explanation>', text, re.DOTALL)
    
    predicted_answer = int(answer_match.group(1)) if answer_match else None
    explanation = explanation_match.group(1).strip() if explanation_match else ""
    
    return predicted_answer, explanation

def generate_response(model, tokenizer, question, system_prompt, device="cuda"):
    """Generate model response for a question"""
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question},
    ]
    
    text = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False,
    )
    
    FastLanguageModel.for_inference(model)
    inputs = tokenizer(text, return_tensors="pt").to(device)
    
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.1,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    result = tokenizer.decode(output[0][len(inputs.input_ids[0]):], skip_special_tokens=True)
    return result

def format_question(example):
    """Format a dataset example into question format"""
    question = f"""{example["question"]}
Options:
0. {example["opa"]}
1. {example["opb"]}
2. {example["opc"]}
3. {example["opd"]}"""
    return question

def evaluate_model(model, tokenizer, system_prompt, num_samples=10):
    """Evaluate model on MedMCQA validation set"""
    # Load validation dataset
    dataset = load_dataset("openlifescienceai/medmcqa", split="validation")
    
    # Take first num_samples for evaluation
    eval_samples = dataset.select(range(num_samples))
    
    results = []
    correct_predictions = 0
    
    print("Evaluating model...")
    print("=" * 80)
    
    for i, example in enumerate(eval_samples):
        question = format_question(example)
        correct_answer = example["cop"]
        
        # Generate response
        response = generate_response(model, tokenizer, question, system_prompt)
        
        # Extract answer and explanation
        predicted_answer, explanation = extract_answer_and_explanation(response)
        
        # Check accuracy
        is_correct = predicted_answer == correct_answer
        if is_correct:
            correct_predictions += 1
        
        # Store result
        result = {
             'question_id': i,
             'question': example["question"],
             'correct_answer': correct_answer,
             'predicted_answer': predicted_answer,
             'is_correct': is_correct,
             'explanation': explanation,
             'full_response': response
         }
        results.append(result)
        
        # Print individual result
        print(f"Question {i+1}/{num_samples}")
        print(f"Q: {example['question']}")
        print(f"Correct: {correct_answer}, Predicted: {predicted_answer}, {'✓' if is_correct else '✗'}")
        print(f"Response: {response}")
        print("-" * 40)
    
    # Calculate metrics
    accuracy = correct_predictions / num_samples
    
    # Summary
    print("\nEVALUATION SUMMARY")
    print("=" * 50)
    print(f"Total samples: {num_samples}")
    print(f"Correct predictions: {correct_predictions}")
    print(f"Accuracy: {accuracy:.2%}")
    
    # Format validation
    format_valid_count = sum(1 for r in results if r['predicted_answer'] is not None)
    format_accuracy = format_valid_count / num_samples
    print(f"Format compliance: {format_accuracy:.2%} ({format_valid_count}/{num_samples})")
    
    # Show some examples
    print(f"\nSample predictions:")
    for i in range(min(3, num_samples)):
        r = results[i]
        print(f"Q: {r['question'][:60]}...")
        print(f"A: {r['predicted_answer']} ({'Correct' if r['is_correct'] else 'Wrong'})")
        print(f"Explanation: {r['explanation'][:80]}...")
        print()
    
    return results, accuracy

In [61]:
results, accuracy = evaluate_model(model, tokenizer, system_prompt, num_samples=10)

Evaluating model...
Question 1/10
Q: Which of the following is not true for myelinated nerve fibers:
Correct: 0, Predicted: 0, ✓
Response: 0</answer><explanation>Ans. is 'a' i.e., Impulse through myelinated fibers is slower than non-myelinated fibers</explanation>
----------------------------------------
Question 2/10
Q: Which of the following is not true about glomerular capillaries')
Correct: 0, Predicted: 3, ✗
Response: 3</answer><explanation>Glomerular capillaries are continuous with the afferent and efferent arterioles. The blood flow through the glomerular capillaries is constant. The blood flow is regulated by constriction or dilation of the afferent arteriole. The oncotic pressure of the fluid leaving the capillaries is greater than that of fluid entering it. The concentration of glucose in the capillaries is greater than that in the glomerular filtrate. The hematocrit of the fluid leaving the capillaries is greater than that of the fluid entering it.</explanation>
------------